In [1]:
import os
import re
import cv2
import numpy as np

In [2]:
def extract_video_frames(path_in):
    
    # Extract name of the video file and create a new directory to store the extracted frames
    video_name = path_in[:path_in.rfind(".")].strip()
    video_name = "{} frames".format(video_name)
    os.mkdir(video_name)
    path_out = video_name

    # Initialize video capture object
    vidcap = cv2.VideoCapture(path_in) 

    count = 0
    success = True

    # Loop through frames in the video file and extract each frame as a PNG image file
    while success:
        # Set the position of the video capture object to the current frame
        vidcap.set(cv2.CAP_PROP_POS_MSEC,(count*5000))
        # Read the frame and check if the read was successful
        success,image = vidcap.read() 

        if not success:
            # If the read was not successful, print the number of frames read and exit the loop
            print("{} frames read.".format(count))
            break

        # Save the current frame as a PNG image file
        cv2.imwrite(os.path.join(path_out, "frame%d.png" % count), image) 
        count = count + 1

    # Return the path to the directory containing the extracted frames
    return path_out

In [3]:
def make_barcode(frames_dir, bar_width = 1, dims = None):
    
    # Get a list of all the image files in the given directory and sort them
    frame_files = sorted([f for f in os.listdir(os.path.join(frames_dir)) if f.endswith('.png') or f.endswith('.jpg')])
    
    # Sort the list of image files by their numeric index in the filename
    frame_files.sort(key = lambda f: int(re.sub('\D', '', f)))
    
    # Calculate the dimensions of the resulting barcode image
    barcode_width = bar_width * len(frame_files)
    barcode_height = cv2.imread(os.path.join(frames_dir, frame_files[0]), cv2.IMREAD_UNCHANGED).shape[0]
    
    # Create an empty numpy array to hold the barcode image
    barcode = np.zeros((barcode_height, barcode_width, 3), np.uint8)
    
    # Loop through the image files and add them to the barcode image
    for i, f in zip(range(0, barcode_width, bar_width), frame_files):
       
        frame_path = os.path.join(frames_dir, f) # Load the image file
        frame = cv2.imread(frame_path, cv2.IMREAD_UNCHANGED)
        
        # Resize the image to the desired width and add it to the barcode image
        bar = cv2.resize(frame, (bar_width, barcode_height), interpolation = cv2.INTER_AREA)
        barcode[:,i:(i+bar_width)] = bar

    # Set the output filename for the barcode image
    filename = os.path.splitext(os.path.basename(frames_dir))[0] + "-5s-barcode.png"
    
    # Save the barcode image to disk and display it
    cv2.imwrite(filename, barcode)

    # Return the filename of the generated barcode image
    return filename
  

In [4]:
# List of video files
video_files = ["Bliss.mkv", "Nightmares.mkv", "Paradise.mkv", "Coping.mkv", "Dreams.mkv"]

# Loop through each video file
for video_file in video_files:
    # Extract frames from the video
    frames_dir = extract_video_frames(video_file)
    # Generate barcode from the extracted frames
    barcode_file = make_barcode(frames_dir)
    # Print the filename of the generated barcode image
    print("Barcode generated for {}: {}".format(video_file, barcode_file))


131 frames read.
Barcode generated for Bliss.mkv: Bliss frames-5s-barcode.png
133 frames read.
Barcode generated for Nightmares.mkv: Nightmares frames-5s-barcode.png
135 frames read.
Barcode generated for Paradise.mkv: Paradise frames-5s-barcode.png
138 frames read.
Barcode generated for Coping.mkv: Coping frames-5s-barcode.png
138 frames read.
Barcode generated for Dreams.mkv: Dreams frames-5s-barcode.png
